In [1]:

from cplvm import CPLVM
from cplvm import CPLVMLogNormalApprox

2024-04-24 13:52:22.704671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 13:52:25.750609: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-24 13:52:25.750671: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-24 13:52:54.672327: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("data/polyps/SuppTable2_PolypALL_merged.txt.gz",sep="\t",index_col=0)

In [4]:
data = data.T

In [5]:
meta = pd.read_csv("data/polyps/np_alexandria_meta_with_sss.csv",index_col=0 )

In [6]:
no_polyp_order = pd.read_csv("data/polyps/no_polyp_epithelial_pseudotime_order.csv",header=None, index_col=0)

In [7]:
no_polyp_cells = data.loc[no_polyp_order.index]

In [9]:
polyp_order = pd.read_csv("data/polyps/polyp_epithelial_pseudotime_order.csv",header=None, index_col=0)
polyp_cells = data.loc[polyp_order.index]

In [11]:
# grab variable genes from previous preprocessing
import scanpy as sc
adata=sc.read_h5ad("data/polyps/processed_adata.h5ad")

var_genes = adata.var_names

In [17]:
#background_cells = t_milk[t_milk.obs["Sample Timepoint"] == "Post-Dose 3"].obs_names
#foreground_cells = t_milk[t_milk.obs["Sample Timepoint"] == "Post-COVID"].obs_names
X =  data.loc[polyp_order.index,var_genes] #background
Y =  data.loc[no_polyp_order.index,var_genes]

In [18]:
n, m = X.shape[0], Y.shape[0]
assert X.shape[1] == Y.shape[1]
p = X.shape[1]

In [20]:
# Set up CPLVM
cplvm = CPLVM(
    k_shared=5,
    k_foreground=5,
    compute_size_factors=True,
    offset_term=False)
approx_model = CPLVMLogNormalApprox(
    X.T, 
    Y.T, 
    k_shared=5, 
    k_foreground=5, 
    compute_size_factors=True, 
    offset_term=False
)
# Fit model
model_output = cplvm.fit_model_vi(
    X.T,
    Y.T,
    approximate_model=approx_model,
)

2024-04-24 14:03:23.373113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-24 14:03:23.373428: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-24 14:03:23.373457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dev3): /proc/driver/nvidia/version does not exist
2024-04-24 14:03:23.453466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
import numpy as np
# Foreground-specific loadings
W_mean = model_output["approximate_model"].qw_mean.numpy()
W_stddev = model_output["approximate_model"].qw_stddv.numpy()
W = np.exp(W_mean + W_stddev ** 2)
# Shared loadings
S_mean = model_output["approximate_model"].qs_mean.numpy()
S_stddev = model_output["approximate_model"].qs_stddv.numpy()
S = np.exp(S_mean + S_stddev ** 2)

In [22]:
W_df = pd.DataFrame(W, index=var_genes)

In [23]:
W_df.sort_values(0,ascending=False).head(20)


,0,1,2,3,4
EPAS1,37.011562,2.505452,11.588847,3.281975,1.563567
CLCA4,6.527321,0.114290,0.805846,0.062172,0.027563
TMEM213,5.379430,0.261537,1.958501,0.481376,2.113334
KRT15,3.065575,1.035643,1.657865,9.654953,0.020057
ZG16B,2.525077,0.952249,5.355985,0.850873,1.270524
ALOX15,2.229563,0.146258,8.655851,0.146958,0.458111
PTGS2,2.193851,1.314303,0.149214,1.386241,0.560061
FGFR3,1.995051,0.215648,0.270904,0.333844,0.033372
EGR3,1.782765,1.083810,0.713113,0.850476,0.148539
GOLGA8A,1.567339,0.236442,0.087709,0.241319,0.061863


In [24]:
W_df.sort_values(1,ascending=False).head(20)


,0,1,2,3,4
IL8,0.140932,15.617467,0.085721,0.071513,2.470618
CXCL1,0.169260,8.026592,1.070843,1.234855,2.020879
CXCL2,0.173950,5.356677,0.060546,0.206176,0.180835
MT2A,0.898122,3.940605,4.551134,7.393543,1.293640
EPAS1,37.011562,2.505452,11.588847,3.281975,1.563567
IGFBP5,0.433175,1.722938,0.999126,0.695058,0.168399
CXCL3,0.652706,1.601211,0.168461,0.565463,0.343491
STATH,1.245868,1.435694,58.936340,14.044562,2.355413
PTGS2,2.193851,1.314303,0.149214,1.386241,0.560061
C8orf4,0.397531,1.194166,9.640998,1.063897,1.229327


In [25]:
W_df.sort_values(2,ascending=False).head(20)


,0,1,2,3,4
STATH,1.245868,1.435694,58.936340,14.044562,2.355413
S100A4,0.617788,0.341758,33.211906,0.134784,5.642163
S100A9,0.074754,0.046516,14.944301,0.432727,0.130210
LYZ,0.464671,0.799435,13.871137,1.161346,4.839598
EPAS1,37.011562,2.505452,11.588847,3.281975,1.563567
GDF15,1.173148,0.253350,10.059739,0.255070,1.989056
S100P,0.122791,0.258034,9.719593,0.069894,14.814454
C8orf4,0.397531,1.194166,9.640998,1.063897,1.229327
ALOX15,2.229563,0.146258,8.655851,0.146958,0.458111
CTD-2319I12.1,0.135030,0.340287,6.381184,0.076191,4.934481


In [26]:
W_df.sort_values(3,ascending=False).head(20)


,0,1,2,3,4
STATH,1.245868,1.435694,58.936340,14.044562,2.355413
KRT15,3.065575,1.035643,1.657865,9.654953,0.020057
SPINK5,1.072831,0.305639,0.702152,7.651138,0.119192
MT2A,0.898122,3.940605,4.551134,7.393543,1.293640
EPAS1,37.011562,2.505452,11.588847,3.281975,1.563567
FN1,0.076397,0.063284,0.047646,3.103796,0.018876
HIST1H4C,0.488935,0.214067,1.285653,2.770895,0.146848
CTGF,0.175279,0.311428,0.090343,1.967263,0.232510
SULT1E1,1.196201,0.588129,0.095512,1.754593,0.050879
STMN1,0.392167,0.380482,0.103836,1.504647,0.029650


In [27]:
W_df.sort_values(4,ascending=False).head(20)


,0,1,2,3,4
MSMB,0.201846,0.086526,0.073033,0.449068,66.229393
SCGB1A1,0.234297,0.145533,0.096655,0.039343,65.787979
BPIFB1,0.453028,0.212194,0.512164,0.313111,57.918468
LYPD2,0.032810,0.203982,0.762342,0.148474,17.828484
BPIFA1,0.160858,0.085168,0.228642,0.132394,14.838365
S100P,0.122791,0.258034,9.719593,0.069894,14.814454
PSCA,0.053289,0.126629,6.185560,0.199318,14.482382
LCN2,0.394516,0.493732,1.299837,0.091336,13.973104
RARRES1,0.116623,0.464661,0.508865,0.207227,11.392974
MUC5AC,0.077663,0.062764,0.087646,0.067710,9.282598


In [28]:
W_df.to_csv("data/polyps/CPLVM_W_foregroundloadings.csv")